<a href="https://colab.research.google.com/github/spatiebalk/text_mining_project/blob/master/text_generation_GRU_hp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text generation GRU with Harry Potter text


In [16]:
import numpy as np
import tensorflow as tf
import os
from os.path import join, isfile
import time
import csv
from tqdm import tqdm
import json
from os import listdir
from os.path import isfile, join
import sys

In [17]:
import sys
print(sys.version)
print(tf.__version__)

3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]
2.4.0


In [18]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


### Harry Potter data

In [19]:
text = open('/content/gdrive/My Drive/TxMM/harrypotter.txt').read()
print(len(text))

6251649


### 1. GRU
From https://medium.com/towards-artificial-intelligence/create-your-own-harry-potter-short-story-using-rnn-and-tensorflow-853b3ed1b8f3 and https://www.tensorflow.org/tutorials/text/text_generation

In [20]:
# print(text[:200])

In [21]:
vocab = sorted(set(text))
char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)
text_as_int = np.array([char2index[c] for c in text])

In [22]:
seq_length = 50
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(data):
  input_text = data[:-1]
  target_text = data[1:]
  return input_text, target_text

dataset = sequences.map(split_input_target)

BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [23]:
vocab_size = len(vocab)
embedding_dim = 300

# Number of RNN units 
rnn_units1 = 512
rnn_units2 = 256
rnn_units= [rnn_units1, rnn_units2]

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
       batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units1, return_sequences=True,
       stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.GRU(rnn_units2, return_sequences=True,
       stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(
vocab_size = vocab_size,
embedding_dim=embedding_dim,
rnn_units=rnn_units,
batch_size=BATCH_SIZE)

def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels,
         logits, from_logits=True)
  
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (64, None, 300)           31800     
_________________________________________________________________
gru_4 (GRU)                  (64, None, 512)           1250304   
_________________________________________________________________
gru_5 (GRU)                  (64, None, 256)           591360    
_________________________________________________________________
dense_2 (Dense)              (64, None, 106)           27242     
Total params: 1,900,706
Trainable params: 1,900,706
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/gdrive/My Drive/TxMM/GRU_results/training_checkpoints_GRU_hp'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
   filepath=checkpoint_prefix, save_weights_only=True)

In [25]:
EPOCHS= 50
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])
latest_check = tf.train.latest_checkpoint(checkpoint_dir)

Epoch 1/50
1915/1915 [==============================] - 46s 23ms/step - loss: 1.9441 - accuracy: 0.4579
Epoch 2/50
1915/1915 [==============================] - 45s 23ms/step - loss: 1.3308 - accuracy: 0.6034
Epoch 3/50
1915/1915 [==============================] - 45s 23ms/step - loss: 1.2778 - accuracy: 0.6172
Epoch 4/50
1915/1915 [==============================] - 45s 23ms/step - loss: 1.2527 - accuracy: 0.6237
Epoch 5/50
1915/1915 [==============================] - 43s 22ms/step - loss: 1.2373 - accuracy: 0.6279
Epoch 6/50
1915/1915 [==============================] - 43s 22ms/step - loss: 1.2278 - accuracy: 0.6305
Epoch 7/50
1915/1915 [==============================] - 43s 22ms/step - loss: 1.2203 - accuracy: 0.6323
Epoch 8/50
1915/1915 [==============================] - 43s 22ms/step - loss: 1.2153 - accuracy: 0.6340
Epoch 9/50
1915/1915 [==============================] - 43s 22ms/step - loss: 1.2111 - accuracy: 0.6349
Epoch 10/50
1915/1915 [==============================] - 43s 22m

In [26]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(latest_check)
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (1, None, 300)            31800     
_________________________________________________________________
gru_6 (GRU)                  (1, None, 512)            1250304   
_________________________________________________________________
gru_7 (GRU)                  (1, None, 256)            591360    
_________________________________________________________________
dense_3 (Dense)              (1, None, 106)            27242     
Total params: 1,900,706
Trainable params: 1,900,706
Non-trainable params: 0
_________________________________________________________________


In [27]:
def generate_text(model, start_string):

  num_generate = 1000  #can be anything you like
  input_eval = [char2index[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)
  text_generated = []
  scaling = 0.5 #kept at a lower value here
  # Here batch size == 1
  model.reset_states()

  for i in range(num_generate):
    predictions = model(input_eval)
    # remove the batch dimension
    predictions = tf.squeeze(predictions, 0)

    # using a categorical distribution to predict the character returned by the model
    predictions = predictions / scaling
    predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

    # Pass the predicted character as the next input to the model
    # along with the previous hidden state
    input_eval = tf.expand_dims([predicted_id], 0)

    text_generated.append(index2char[predicted_id])

  return (start_string + "".join(text_generated))

In [28]:
print(generate_text(model, start_string= "Severus Snape"))

Severus Snape to be an an indeather to his wand was all, the wand shouting the golden tiny of the office of the counter and had said behind the wand. He could think of the softly who closed to come has he saw that a corner and looked at the darkness upon his head, and he could have told her wand.
"I don’t think what it was a second of the back to the last that it was entered it, and he had the creak a little of the room, and a creath and tried to be some of the air of the staircase, and Harry was a silent to the and the wand had been stocked the whole the car so the part of the window to the door be the sword in the castle on the sign in the silver closed his wand to the concealed the house and felt he had been at his finger of the trait. "I don’t realized his last of the staircase all the tiny careft that I can one himself of the second in a moment that he was the very wand that he had took a long little side of there the possible and contruffled the contactly place to be containly re

In [29]:
# open prompts file
prompts = open('/content/gdrive/My Drive/TxMM/hp_prompts.txt', 'r').readlines()
generated_dict = {}

for i, p in tqdm(enumerate(prompts)):
  # generate 1000 characters
  gen_text = generate_text(model, start_string=p)
  generated_dict[i] = gen_text

with open('/content/gdrive/My Drive/TxMM/GRU_results/hp_generated_texts.json', 'w') as fp:
  json.dump(generated_dict, fp)


0it [00:00, ?it/s]
1it [00:06,  6.30s/it]
2it [00:12,  6.28s/it]
3it [00:18,  6.29s/it]
4it [00:25,  6.31s/it]
5it [00:31,  6.29s/it]
6it [00:37,  6.27s/it]
7it [00:43,  6.24s/it]
8it [00:49,  6.22s/it]
9it [00:56,  6.20s/it]
10it [01:02,  6.19s/it]
11it [01:08,  6.17s/it]
12it [01:14,  6.15s/it]
13it [01:20,  6.14s/it]
14it [01:26,  6.13s/it]
15it [01:33,  6.29s/it]
16it [01:39,  6.25s/it]
17it [01:45,  6.23s/it]
18it [01:51,  6.20s/it]
19it [01:58,  6.22s/it]
20it [02:04,  6.32s/it]
21it [02:11,  6.35s/it]
22it [02:17,  6.30s/it]
23it [02:23,  6.24s/it]
24it [02:29,  6.19s/it]
25it [02:35,  6.17s/it]
26it [02:41,  6.15s/it]
27it [02:47,  6.13s/it]
28it [02:53,  6.12s/it]
29it [03:00,  6.13s/it]
30it [03:06,  6.13s/it]
31it [03:12,  6.10s/it]
32it [03:18,  6.10s/it]
33it [03:24,  6.09s/it]
34it [03:30,  6.08s/it]
35it [03:36,  6.08s/it]
36it [03:42,  6.09s/it]
37it [03:48,  6.08s/it]
38it [03:54,  6.06s/it]
39it [04:00,  6.06s/it]
40it [04:06,  6.05s/it]
41it [04:12,  6.06s/it]
42it 